<a href="https://colab.research.google.com/github/Niv0902/Shablool/blob/main/Tut8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": [],
      "private_outputs": true
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    },
    "accelerator": "GPU"
  },
  "cells": [
    {
      "cell_type": "markdown",
      "source": [
        "# Generates images from text prompts.\n",
        "\n",
        "By Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings). It uses a 602M parameter diffusion model trained on Conceptual 12M. See the GitHub repo for more information: https://github.com/crowsonkb/v-diffusion-pytorch."
      ],
      "metadata": {
        "id": "sThFl4fJtEQJ"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "# @title Licensed under the MIT License\n",
        "\n",
        "# Copyright (c) 2022 Katherine Crowson\n",
        "\n",
        "# Permission is hereby granted, free of charge, to any person obtaining a copy\n",
        "# of this software and associated documentation files (the \"Software\"), to deal\n",
        "# in the Software without restriction, including without limitation the rights\n",
        "# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell\n",
        "# copies of the Software, and to permit persons to whom the Software is\n",
        "# furnished to do so, subject to the following conditions:\n",
        "\n",
        "# The above copyright notice and this permission notice shall be included in\n",
        "# all copies or substantial portions of the Software.\n",
        "\n",
        "# THE SOFTWARE IS PROVIDED \"AS IS\", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR\n",
        "# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,\n",
        "# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE\n",
        "# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER\n",
        "# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,\n",
        "# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN\n",
        "# THE SOFTWARE."
      ],
      "metadata": {
        "cellView": "form",
        "id": "eb9FKu1rtH6R"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "xa4Er6nMgI1b"
      },
      "outputs": [],
      "source": [
        "# Check the GPU\n",
        "\n",
        "!nvidia-smi"
      ]
    },
    {
      "cell_type": "markdown",
      "source": [
        "### Install dependencies (no need to rerun this section if you restart the notebook runtime)"
      ],
      "metadata": {
        "id": "0XknUUiEsPCL"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "# Install dependencies\n",
        "\n",
        "!git clone --recursive https://github.com/crowsonkb/v-diffusion-pytorch\n",
        "%pip install git+https://github.com/crowsonkb/k-diffusion\n",
        "%pip install numpy==1.23.5"
      ],
      "metadata": {
        "id": "xQVAY5-4g8ta"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "If the \"Restart runtime\" button appears above, please click it to restart the runtime, then continue below."
      ],
      "metadata": {
        "id": "Q7gmndIVKGbQ"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "# Download the diffusion model\n",
        "# SHA-256: 4fc95ee1b3205a3f7422a07746383776e1dbc367eaf06a5b658ad351e77b7bda\n",
        "\n",
        "!mkdir v-diffusion-pytorch/checkpoints\n",
        "!curl -L --http1.1 https://the-eye.eu/public/AI/models/v-diffusion/cc12m_1_cfg.pth > v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth"
      ],
      "metadata": {
        "id": "9410kZ2aipVg"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "### Import modules and load models"
      ],
      "metadata": {
        "id": "tbjtOS68sazS"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "# Imports\n",
        "\n",
        "import gc\n",
        "import math\n",
        "import sys\n",
        "\n",
        "import clip\n",
        "from IPython import display\n",
        "import k_diffusion as K\n",
        "import torch\n",
        "from torch import nn\n",
        "from torchvision import utils\n",
        "from torchvision.transforms import functional as TF\n",
        "from tqdm.notebook import trange, tqdm\n",
        "\n",
        "sys.path.append('/content/v-diffusion-pytorch')\n",
        "\n",
        "from diffusion import get_model"
      ],
      "metadata": {
        "id": "f_6OhF85i5vp"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "# Load the models\n",
        "!mkdir -p v-diffusion-pytorch/checkpoints\n",
        "!wget https://huggingface.co/multimodalart/crowsonkb-v-diffusion-cc12m-1-cfg/resolve/main/cc12m_1_cfg.pth -O v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth\n",
        "\n",
        "inner_model = get_model('cc12m_1_cfg')()\n",
        "_, side_y, side_x = inner_model.shape\n",
        "inner_model.load_state_dict(torch.load('v-diffusion-pytorch/checkpoints/cc12m_1_cfg.pth', map_location='cpu',weights_only=False))\n",
        "inner_model = inner_model.half().cuda().eval().requires_grad_(False)\n",
        "model = K.external.VDenoiser(inner_model)\n",
        "clip_model = clip.load(inner_model.clip_model, jit=False, device='cpu')[0]"
      ],
      "metadata": {
        "id": "v7Zl0mg1jrBP"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "#@title Settings\n",
        "\n",
        "#@markdown The text prompt\n",
        "prompt = 'New York City, oil on canvas'  #@param {type:\"string\"}\n",
        "\n",
        "#@markdown The strength of the text conditioning (0 means don't condition on text, 1 means sample images that match the text about as well as the images match the text captions in the training set, 3+ is recommended).\n",
        "weight = 5  #@param {type:\"number\"}\n",
        "\n",
        "#@markdown Sample this many images.\n",
        "n_images = 4  #@param {type:\"integer\"}\n",
        "\n",
        "#@markdown Specify the number of diffusion timesteps (default is 50, can lower for faster but lower quality sampling).\n",
        "steps =   50#@param {type:\"integer\"}\n",
        "\n",
        "#@markdown The random seed. Change this to sample different images.\n",
        "seed = 0  #@param {type:\"integer\"}\n",
        "\n",
        "#@markdown Display progress every this many timesteps.\n",
        "display_every =   10#@param {type:\"integer\"}"
      ],
      "metadata": {
        "cellView": "form",
        "id": "kitsmKsekOJ_"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "source": [
        "### Actually do the run..."
      ],
      "metadata": {
        "id": "kuTFyUdbrlcK"
      }
    },
    {
      "cell_type": "code",
      "source": [
        "target_embed = clip_model.encode_text(clip.tokenize(prompt)).float().cuda()\n",
        "\n",
        "\n",
        "class CFGDenoiser(nn.Module):\n",
        "    def __init__(self, model, cond_scale):\n",
        "        super().__init__()\n",
        "        self.inner_model = model\n",
        "        self.cond_scale = cond_scale\n",
        "\n",
        "    def forward(self, x, sigma, clip_embed):\n",
        "        x_in = torch.cat([x] * 2)\n",
        "        sigma_in = torch.cat([sigma] * 2)\n",
        "        clip_embed_in = torch.cat([torch.zeros_like(clip_embed), clip_embed])\n",
        "        uncond, cond = self.inner_model(x_in, sigma_in, clip_embed=clip_embed_in).chunk(2)\n",
        "        return uncond + (cond - uncond) * self.cond_scale\n",
        "\n",
        "\n",
        "def callback(info):\n",
        "    if info['i'] % display_every == 0:\n",
        "        nrow = math.ceil(info['denoised'].shape[0] ** 0.5)\n",
        "        grid = utils.make_grid(info['denoised'], nrow, padding=0)\n",
        "        tqdm.write(f'Step {info[\"i\"]} of {steps}, sigma {info[\"sigma\"]:g}:')\n",
        "        display.display(K.utils.to_pil_image(grid))\n",
        "        tqdm.write(f'')\n",
        "\n",
        "\n",
        "def run():\n",
        "    gc.collect()\n",
        "    torch.cuda.empty_cache()\n",
        "    torch.manual_seed(seed)\n",
        "    sigmas = K.sampling.get_sigmas_karras(steps, 1e-2, 160, device='cuda')\n",
        "    x = torch.randn([n_images, 3, side_y, side_x], device='cuda') * sigmas[0]\n",
        "    model_wrap = CFGDenoiser(torch.cuda.amp.autocast()(model), weight)\n",
        "    extra_args = {'clip_embed': target_embed.repeat([n_images, 1])}\n",
        "    outs = K.sampling.sample_lms(model_wrap, x, sigmas, extra_args=extra_args, callback=callback)\n",
        "    tqdm.write('Done!')\n",
        "    for i, out in enumerate(outs):\n",
        "        filename = f'out_{i}.png'\n",
        "        K.utils.to_pil_image(out).save(filename)\n",
        "        display.display(display.Image(filename))\n",
        "\n",
        "\n",
        "run()"
      ],
      "metadata": {
        "id": "MnjPM_PUkYui"
      },
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [],
      "metadata": {
        "id": "JP7fqew6jwK5"
      },
      "execution_count": null,
      "outputs": []
    }
  ]
}

NameError: name 'null' is not defined